In [ ]:
%run ../Helper/Setup.ipynb

In [ ]:
def import_csv(file, group=True):
    df = pd.read_csv(file)
    
    if group:
        df = group_data(df)
    
    return df

def group_data(df):
    filtered = df.groupby(by=['server', 'client'], as_index=False).agg({'goodput': ['mean', 'std'], 'repetition': 'count'})
    filtered.columns = list(map(''.join, filtered.columns.values))
    return filtered

In [ ]:
client_timestamps_goracle = import_csv('../../results/baseline/client-timestamps/goracle/processed.csv')
client_timestamps_goracle_optimized = import_csv('../../results/baseline/client-timestamps/goracle-optimized/processed.csv')

In [ ]:
def plot_heatmap(data=None, title=None):
    
    implementation_order = {
        'lsquic': 6,
        'lsquic-goracle': 6,
        'quiche': 5,
        'quiche-goracle': 5,
        'aioquic': 1,
        'mvfst': 3,
        'quic-go': 2,
        'picoquic': 4,
    }
    
    data = data.sort_values(by=['client'], key=lambda xs: list(map(lambda x: implementation_order.get(x), xs)))
    data = data.sort_values(by=['server'], key=lambda xs: list(map(lambda x: implementation_order.get(x), xs)))

        
    goodput_grid = pd.pivot_table(
        data,
        values='goodputmean',
        index=['client'],
        columns = ['server'],
    )
    
    goodput_grid = goodput_grid.sort_values(by=['client'], key=lambda xs: list(map(lambda x: -implementation_order.get(x), xs)))
    goodput_grid = goodput_grid.sort_values(by=['server'], axis=1, key=lambda xs: list(map(lambda x: implementation_order.get(x), xs)))

        

    implementation_names = {
        'lsquic': 'LSQUIC',
        'lsquic-goracle': 'LSQUIC',
        'quiche': 'quiche',
        'quiche-goracle': 'quiche',
        'aioquic': 'aioquic',
        'mvfst': 'mvfst',
        'quic-go': 'quic-go',
        'picoquic': 'picoquic',
    }
    
    colors = [ColorDarkerBlue, '#AAAAAA', ColorRed]
    # Set your custom color palette
    cmap = sns.color_palette(f"blend:{','.join(colors)}", as_cmap=True)
    
    width, height = set_size('single')
    
    fig, ax = plt.subplots(figsize=(width, height*1))
        
    sns.heatmap(
        goodput_grid,
        linewidth=1,
        #cmap = 'coolwarm',
        cmap = cmap,
        annot=True, fmt=".0f",
        cbar_kws={'label': 'Goodput in Mbit/s'},
        ax=ax,
        annot_kws={'fontsize': 6,},
    )
    
    ax.set_xlabel('Server')
    ax.set_ylabel('Client')
    
    ax.set_xticklabels(list(map(lambda x: implementation_names.get(x.get_text()), ax.get_xticklabels())))
    ax.set_yticklabels(list(map(lambda x: implementation_names.get(x.get_text()), ax.get_yticklabels())))
    
    
    ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='center')
    ax.set_yticklabels(ax.get_yticklabels(), rotation=30, ha='right')
    fig.tight_layout()
    
    return fig, ax

def plot_heatmap_full(data=None, file=None, title=None):
    
    if file:
        data = import_csv(file)
        data.sort_values(by=['server','client'], inplace=True)
        data = group_data(data)
    
    goodput_grid = pd.pivot_table(
        data,
        values='goodputmean',
        index=['client'],
        columns = ['server'],
    )

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    sns.heatmap(
        goodput_grid,
        linewidth=1,
        #cmap = 'coolwarm',
        annot=True, fmt=".0f",
        cbar_kws={'label': 'Goodput in Mbit/s'},
        ax=axes[0],
    )
    
    std_grid = pd.pivot_table(
        data,
        values='goodputstd',
        index=['client'],
        columns = ['server'],
    )
    sns.heatmap(
        std_grid,
        linewidth=3,
        cmap = 'coolwarm',
        annot=True, fmt=".0f",
        cbar_kws={'label': 'Std'},
        ax=axes[1],
    )
    
    sns.heatmap(
        std_grid / goodput_grid * 100,
        linewidth=3,
        cmap = 'coolwarm',
        annot=True, fmt=".1f",
        cbar_kws={'label': 'Std / Mean * 100 %'},
        ax=axes[2],
    )
    
    if title:
        fig.suptitle(title)
    

In [ ]:
# Figures in the paper
fig, axes = plot_heatmap(client_timestamps_goracle)
fig.savefig('baseline.pdf')

fig, axes = plot_heatmap(client_timestamps_goracle_optimized)
fig.savefig('baseline-optimized.pdf')

In [ ]:
plot_heatmap_full(client_timestamps_goracle, title='Client Timestamp Goracle')
plot_heatmap_full(client_timestamps_goracle_optimized, title='Client Timestamp Goracle Optimized')